In [11]:
import numpy as np
import pandas as pd

In [12]:
def calculation_helper(group_data):
    '''
    Returns the expected return and the BETA. This is
    a helper function that does the calculation for the 
    CAPM formula.
    
    calculation_helper: DataFrame (grouped) -> DataFrame
    
    Requries:
        - CAPM-beta is Cov(R_i-Rf, R_mkt – Rf)/Var(R_mkt-Rf).
        - The consumed data must be merged_data.groupby("CUSIP").
        - The consumed data must contain columns: risk premium (security) and 
        'market risk premium', which are essential for calculations. 
    '''
    # Prechecked in the CAMP_beta function that the columns exist 
    Cov = group_data['risk premium (security)'].cov(group_data['market risk premium'])
    Var = group_data['market risk premium'].var()
    
    # Prechecked that Var do not contain 0 -> Division by zero will not occur 
    beta = Cov / Var 
    exp_return = group_data['RET'].mean()

    return pd.Series({'Expected': exp_return, 'beta': beta})

def CAPM_beta(return_path, factors_path):
    '''
    Returns a dataframe of which each row correspond
    with 1 CUSIP, its expected return and its CAPM-beta.
    
    Str Str -> DataFrame.
    
    Requires:
        - CAPM-beta is Cov(R_i-Rf, R_mkt – Rf)/Var(R_mkt-Rf).
        - the size of the output should be 3225 rows × 3 columns.
    '''
    # returns have year and months separated.
    returns = pd.read_csv(return_path)
    # factors need to separate and unify the units.
    factors = pd.read_csv(factors_path)
    
    #Unifying the units: turning factors into decimals
    factors[['Mkt-RF', 'RF']] = factors[['Mkt-RF', 'RF']] / 100
    factors['year'] = factors['date'] // 100
    factors['month'] = factors['date'] % 100
    
    # calculating the risk premium for individual assets & market risk premium
    merged_data = pd.merge(factors, returns, on = ['year','month'], how = 'inner')
    merged_data['risk premium (security)'] = merged_data['RET'] - merged_data['RF']
    merged_data['market risk premium'] = merged_data['Mkt-RF']
    
    # applying the helper function to calculate the Covariance & Beta.
    result_df = merged_data.groupby("CUSIP").apply(calculation_helper)
    output = pd.DataFrame(data = {
        "CUSIP" : result_df.index,
        "EXP_RETURN" : result_df['Expected'],
        "BETA" : result_df['beta'],
    })
    
    return output

CAPM_beta("A4monthlycrsp.csv", "F-F_Research_Data_Factors.CSV")


,CUSIP,EXP_RETURN,BETA
CUSIP,,,
00081T10,00081T10,0.016252,2.591073
00108M10,00108M10,0.011101,0.741057
00130H10,00130H10,0.016152,1.457100
00163U10,00163U10,0.015538,0.975530
00182C10,00182C10,0.014772,1.406085
...,...,...,...
9959910,9959910,0.009933,0.763107
9972410,9972410,0.012866,1.280548
9972510,9972510,0.017177,1.065298


In [14]:
def slope(data):
    '''
    Returns the slope of the security market line 
    by consuming the output of CAMP_beta. 
    
    slope: DataFrame -> Float
    
    Recall: {SML} :E(R_{i})=R_{f}+\beta _{i}[E(R_{M})-R_{f}]
    
    Requires:
        - The return value is around 0.007981588120535642
    '''
    #print(data.isna().any()) # There is no Missing values (NAN to Drop). 
    
    # The Slop is E[R_(Mkt) - RF] -> OUR GOAL. 
    # The Y-Axis Represents the Expected Return
    Y = data['EXP_RETURN'].values
    # The X-Axis Represents the Systematic Risk (measured by beta)
    X = data['BETA'].values
    slope, intercept = np.polyfit(X, Y, 1)

    return slope

CAMP_beta_output = CAPM_beta("A4monthlycrsp.csv", "F-F_Research_Data_Factors.CSV")
slope(CAMP_beta_output)
print("The Slope is {}".format(slope(CAMP_beta_output)))

The Slope is 0.004773236586034268
